# Converting the queries to flask-sqlalchemy

In [2]:
import sys
import sys
sys.path.append('/Users/shayneufeld/GitHub/parle')
from models import db
from models import *
import numpy as np
import pandas as pd

/Users/shayneufeld/anaconda/lib/python3.5/site-packages/flask_sqlalchemy/__init__.py:839: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


# get_pol_json 
def get_pol_json(politician_id):
    """
    Returns all the data related to a single specific politician when a user requests their profile.
    Anything we want in the profile has to be sent from here in JSON, so this is where all the magic happens.

    Related React.js component: <ProfileBox />

    :param politician_id: id of the single politician requested
    :return: a jsonify'd dictionary with all the desired profile information
    """
    # Get the cursor
    cursor = get_cursor()

    # Build our select query using data from bills_membervote, bills_votequestion, and bills_bill
    # We are currently only interested in the final vote, so the bill description is used to identify those
    # Add any parameters using (%s) or (%(name)s). More information below.
    query = (
        "SELECT m.votequestion_id,m.vote,b.number,b.name_en,b.law,b.short_title_en, v.session_id "
        "FROM bills_membervote m, bills_votequestion v, bills_bill b "
        "WHERE m.politician_id = (%s) "
        "AND v.id = m.votequestion_id "
        "AND v.description_en LIKE (\'%%Bill be now read a third time and do pass%%\') "
        "AND b.id = v.bill_id "
        "ORDER BY v.date DESC"
    )
    # If any parameters are used in the above query, insert them in the parameters tuple
    # Insert parameters in the order used in the query unless using %(name)s placeholders
    # see http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries for more information
    parameters = (politician_id,)

    # Execute query
    cursor.execute(query, parameters)

    # Fetch results
    pol_results = cursor.fetchall()

    # return a JSON response to React (includes header, no extra work needed)
    return jsonify(votes=pol_results)


in postgres DB              | in new db
                            |
1 member vote_question id   | vote.key
2 member vote               | vote.vote
3 bill number               | bill.id or bill.number?
4 bill name                 | bill.slug
5 bill law                  | bill.number
6 bill title                | bill.short_title
7 vote session id           | event.session_id

from votes.mp/votes/1:
    {
      "law": false, 
      "name_en": "An Act to protect insignia of military orders, decorations and medals of cultural significance for future generations", 
      "number": "C-473", 
      "session_id": "40-3", 
      "short_title_en": "Protection of Insignia of Military Orders, Decorations and Medals Act", 
      "vote": "Y", 
      "votequestion_id": 1089
    }, 

In [61]:
def get_pol_json(politician_id):
    #m.votequestion_id,m.vote,b.number,b.name_en,b.law,b.short_title_en, v.session_id
    final_votes = Vote.query.filter_by(member_id=politician_id).filter(Vote.event.has(final=True)).all()
    response_array = []
    for vote in final_votes:
        vote_dictionary = dict()
        vote_dictionary['law'] = vote.event.decision
        vote_dictionary['name_en'] = vote.event.bill.title_en
        vote_dictionary['number'] = vote.event.bill.number
        vote_dictionary['session_id'] = vote.event.bill.session.id
        vote_dictionary['short_name_en'] = vote.event.bill.short_title_en
        vote_dictionary['vote'] = vote.vote
        vote_dictionary['vote_question_id'] = vote.key

        response_array.append(vote_dictionary)
    return response_array

In [218]:
def get_bill_json(bill_id):
    votes = Vote.query.filter(Vote.event.has(final=True))
    parties = []
    for vote in votes:
        parties.append(vote.member.caucus.name)

    parties = np.unique(parties)
    vote_types = ['Yes','No','Abstain']
    
    df = pd.DataFrame(columns=vote_types,index=parties)
    df.loc[:,:] = 0
    for vote in votes:
        if vote.event.bill.id == bill_id:
            party = vote.member.caucus.name
            if vote.vote == 'Yea':
                df.loc[party,'Yes'] += 1
            elif vote.vote == 'Nay':
                df.loc[party,'No'] += 1
            else:
                df.loc[party,'Abstain'] += 1

    for col in df:            
        df.loc['Total',col] = df[col].sum()
    
    return df

In [219]:
test = get_pol_json(1)
test = get_bill_json(9)
test

,Yes,No,Abstain
Bloc Québécois,0,10,0
Conservative,0,90,0
Green Party,0,1,0
Liberal,174,0,0
NDP,0,41,0
Total,174,142,0
